In [2]:
import gc
import joblib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
sys.path.append("../")
import time
import warnings
warnings.simplefilter("ignore")
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
from utils.common import sigmoid, reverse_sigmoid
from utils.constants import *
from utils.extraction_helpers import read_file

In [4]:
%load_ext autoreload
%autoreload

### Gambling13

In [29]:
gambling6 = read_file(f"{SUBMISSION_DATA_PATH}/gambling6_0.801.csv")
# waikey["prediction2"] = read_file(f"{EXP_PATH}/16.lgbm_dart_6666_new/scores_df.csv").mean(axis=1)

Shape of data: (924621, 2)


In [30]:
gambling6["prediction"] = gambling6["prediction"].apply(reverse_sigmoid)

In [31]:
new = read_file(f"{SUBMISSION_DATA_PATH}/first_real_blending_of_dart_and_rnn.csv")

Shape of data: (924621, 2)


In [32]:
new["prediction"] = new["prediction"].apply(reverse_sigmoid)

In [33]:
pd.concat([gambling6["prediction"], new["prediction"]], axis=1).describe()

,prediction,prediction
count,924621.000000,924621.000000
mean,-1.992362,-2.328824
std,2.999725,2.578517
min,-8.275958,-4.422692
25%,-4.334694,-4.372735
50%,-2.842264,-3.908100
75%,0.238266,0.139720
max,7.539622,2.917358


In [25]:
# fig, ax = plt.subplots(figsize=(12, 12))
# # plt.plot(np.arange(0, 1, 0.01), np.arange(0, 1, 0.01), color="orange", linestyle="-")
# # plt.xlabel("WK Blend")
# # plt.ylabel("Chris best NN")
# plt.scatter(gambling6["prediction"], new["prediction"])
# # plt.scatter(s1, s3)
# plt.show()

In [34]:
w = 0.85 * (gambling6["prediction"] - 0.33).apply(sigmoid) + 0.15 * new["prediction"].apply(sigmoid)

In [35]:
sub = pd.read_csv(f"{SUBMISSION_DATA_PATH}/empty.csv")
sub["prediction"] = w
sub.to_csv(f"{SUBMISSION_DATA_PATH}/0.85_gambling6_0.15_blend.csv", index=False)

### Gambling15

In [102]:
chris  = read_file(f"{SUBMISSION_DATA_PATH}/sorted_chris.csv")
chris_best  = read_file(f"{SUBMISSION_DATA_PATH}/chris_best.csv")
waikey = read_file(f"{SUBMISSION_DATA_PATH}/wk_seed_6666.csv")
waikey2 = read_file(f"{SUBMISSION_DATA_PATH}/wk_seed_1020.csv")
ensemble = read_file(f"{SUBMISSION_DATA_PATH}/others_ensemble.csv")
exp_ensemble = read_file(f"{SUBMISSION_DATA_PATH}/others_exponential_ensemble.csv")
blend_trees = read_file(f"{SUBMISSION_DATA_PATH}/others_blend_boosted_tree.csv")

Shape of data: (924621, 2)
Shape of data: (924621, 2)
Shape of data: (924621, 2)
Shape of data: (924621, 2)
Shape of data: (924621, 2)
Shape of data: (924621, 2)
Shape of data: (924621, 2)


In [103]:
chris["prediction"] = 0.5 * chris["prediction"] + 0.5 * chris_best["prediction"]

In [104]:
waikey["sigmoid_prediction"] = (0.5 * waikey["prediction"] + 0.5 * waikey2["prediction"]).apply(sigmoid)

In [105]:
pre_blend_df = pd.DataFrame(
    dict(
        wk=waikey["sigmoid_prediction"],
        chris=chris["prediction"],
        ensemble=ensemble["prediction"],
        exp_ensemble=exp_ensemble["prediction"],
        blend_trees=blend_trees["prediction"]
    )
)

In [106]:
pre_blend_df.describe()

,wk,chris,ensemble,exp_ensemble,blend_trees
count,924621.000000,924621.000000,924621.000000,924621.000000,924621.000000
mean,0.260169,0.312166,0.248046,0.288500,0.348032
std,0.355040,0.383688,0.346811,0.245436,0.381211
min,0.000042,0.000539,0.000052,0.039747,0.013028
25%,0.003120,0.004756,0.002750,0.090925,0.059964
50%,0.029607,0.054550,0.025503,0.188557,0.137505
75%,0.524124,0.711801,0.476758,0.478454,0.622834
max,0.999833,0.999426,0.999906,0.799423,1.190193


In [107]:
mscaler = MinMaxScaler()
pp = mscaler.fit_transform(pre_blend_df)

In [108]:
pre_blend_df = pd.DataFrame(pp, columns=pre_blend_df.columns)

In [109]:
pre_blend_df.describe()

,wk,chris,ensemble,exp_ensemble,blend_trees
count,924621.000000,924621.000000,924621.000000,924621.000000,924621.000000
mean,0.260181,0.311974,0.248030,0.327446,0.284585
std,0.355114,0.384115,0.346861,0.323079,0.323838
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.003078,0.004222,0.002699,0.067369,0.039872
50%,0.029571,0.054071,0.025454,0.195886,0.105743
75%,0.524192,0.712055,0.476776,0.577492,0.518029
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [110]:
pre_blend_df.corr()

,wk,chris,ensemble,exp_ensemble,blend_trees
wk,1.000000,0.977604,0.996852,0.985572,0.995336
chris,0.977604,1.000000,0.970849,0.980694,0.979953
ensemble,0.996852,0.970849,1.000000,0.984296,0.995912
exp_ensemble,0.985572,0.980694,0.984296,1.000000,0.995063
blend_trees,0.995336,0.979953,0.995912,0.995063,1.000000


In [111]:
pre_blend_df["final1"] = (
    0.57 * pre_blend_df["wk"] + 0.17 * pre_blend_df["chris"] + 0.26 * pre_blend_df["ensemble"]
)

In [112]:
sub = pd.read_csv(f"{SUBMISSION_DATA_PATH}/empty.csv")
sub["prediction"] = pre_blend_df["final1"]
sub.to_csv(f"{SUBMISSION_DATA_PATH}/gambling16.csv", index=False)